### Review part [20 Min]:

1. See Sep20 TUT and Sep23 LEC

    1. What is a histogram and what does it describe about the data?

    2. What general conclusions regarding the use of barplots and histograms regarding comparisions sample size considerations can we make from the following examples?

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
pyo.init_notebook_mode()

from scipy import stats
import numpy as np

In [ ]:
# load / reset df
df = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/2e9bd5a67e09b14d01f616b00f7f7e0931515d24/data/2020/2020-07-07/coffee_ratings.csv")
df = df[df['total_cup_points']>65]
df = df[~df['country_of_origin'].isna()]
df = df.rename(columns={'country_of_origin': 'origin', 'total_cup_points': 'points'})

In [ ]:
# fix titles
df.origin = df.origin.str.replace(" (", "<br>(")
df.origin = df.origin.str.replace(", ", ",<br>")

fig = px.histogram(df, x='points', facet_col='origin', 
             facet_col_wrap=6, height=1000, facet_row_spacing=0.05)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("origin=", ""))) # fix titles

In [ ]:
df.origin = df.origin.str.replace("<br>", " ") # fix labels

fig = px.box(df, x='points', y="origin", height=750)

# order plot to be more visually interpretable
fig.update_yaxes(categoryorder='array', 
                 categoryarray=df.groupby("origin")['points'].mean().sort_values().index)

In [ ]:
# add in missing sample sizes
keys = df.origin.value_counts().index.values
vals = df.origin.value_counts().index.values + " (n="+df.origin.value_counts().values.astype(str)+")"
df.origin = df.origin.map({k:v for k,v in zip(keys,vals)})

fig = px.box(df, x='points', y="origin", height=750)
fig.update_yaxes(categoryorder='array', 
                 categoryarray=df.groupby("origin")['points'].mean().sort_values().index)

### Demo [40 minutes]

#### Estimating Averages [20/40 minutes]

1. For which countries above do you think we can most accurately estimate the average "points" score of cups of coffee from a given country? 
2. How does the variability of means of simulated samples change as a function of sample size?
3. Does this seem to change if using (symmetric) `normal`, (skewed) `gamma`, or (other empirical shapes) when using **bootstrapped samples**? 


In [ ]:
# fix titles
df.origin = df.origin.str.replace(" (", "<br>(")
df.origin = df.origin.str.replace(", ", ",<br>")

fig = px.histogram(df, x='points', facet_col='origin', 
                   facet_col_wrap=6, height=1000, facet_row_spacing=0.05)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("origin=", ""))) # fix titles

for i,average in enumerate(dict(df.groupby('origin').points.mean()[df.origin.unique()]).values()):
    fig.add_vline(x=average, line_dash="dot", row=6-int(i/6), col=(1+i)%6)
fig.show()

In [ ]:
# population model
pop_parameter_mu_μ = 0
pop_parameter_sigma_σ = 1
normal_distribution = stats.norm(loc=pop_parameter_mu_μ, scale=pop_parameter_sigma_σ) 

n = 100 # adjust and experiment with this
# np.random.seed(130)
x = normal_distribution.rvs(size=n) # "x" is a sample

print("The sample mean for the current sample is", x.mean()) 
# sample average "x-bar" a (sample) "statistic" (not a parameter)
print(x)
fig = px.histogram(pd.DataFrame({'sampled values': x}), x='sampled values',
                   histnorm='probability density') # so the scale matches the pdf below
fig.add_vline(x=x.mean(), line_dash="dot", annotation_text='Sample mean '+str(x.mean()))

# pdf stands for "probability density function"
support = np.linspace(-4,4,100)
fig.add_trace(go.Scatter(x=support, y=normal_distribution.pdf(support), 
                         mode='lines', name='Poulation Model<br>(normal distribution)'))

In [ ]:
number_of_simulations = 1000 # adjust and experiment with this
simulated_means = np.zeros(number_of_simulations)

# np.random.seed(130) # ?
n = 100 # adjust and experiment with this

for i in range(number_of_simulations):
    # np.random.seed(130) # ?
    simulated_means[i] = stats.norm(loc=0, scale=1).rvs(size=n).mean()

title = str(number_of_simulations)+' simulated means for sample of size n = '+str(n)
fig = px.histogram(pd.DataFrame({title: simulated_means}), x=title,
                   histnorm='probability density')    

support = np.linspace(simulated_means.min(),simulated_means.max(),100)
fig.add_trace(go.Scatter(x=support, y=stats.norm(0,scale=1/np.sqrt(n)).pdf(support), 
                         mode='lines', name='A theoretical<br>distribution of<br>"averages"'))

In [ ]:
# population model
pop_parameter_alpha_α = 2
pop_parameter_theta_θ = 4
gamma_distribution = stats.gamma(a=pop_parameter_alpha_α, scale=pop_parameter_theta_θ)

n = 100 # adjust and experiment with this
# np.random.seed(130)
x = gamma_distribution.rvs(size=n) # "x" is a sample

print("The sample mean for the current sample is", x.mean()) 
# sample average "x-bar" a (sample) "statistic" (not a parameter)
# print(x)

fig = px.histogram(pd.DataFrame({'sampled values': x}), x='sampled values',
                   histnorm='probability density') # so the scale matches the pdf below
fig.add_vline(x=x.mean(), line_dash="dot", annotation_text='Sample mean '+str(x.mean()))

support = np.linspace(0,50,100)
fig.add_trace(go.Scatter(x=support, y=gamma_distribution.pdf(support), 
                         mode='lines', name='Poulation Model<br>(gamma distribution)'))
# pdf stands for "probability density function"

In [ ]:
number_of_simulations = 1000 # adjust and experiment with this
simulated_means = np.zeros(number_of_simulations)

# np.random.seed(130) # ?
n = 100 # adjust and experiment with this

for i in range(number_of_simulations):
    # np.random.seed(130) # ?
    simulated_means[i] = stats.norm(loc=0, scale=1).rvs(size=n).mean()

title = str(number_of_simulations)+' simulated means for sample of size n = '+str(n)
fig = px.histogram(pd.DataFrame({title: simulated_means}), x=title,
                   histnorm='probability density')    

support = np.linspace(simulated_means.min(),simulated_means.max(),100)
fig.add_trace(go.Scatter(x=support, y=stats.norm(0,scale=1/np.sqrt(n)).pdf(support), 
                         mode='lines', name='A theoretical<br>distribution of<br>"averages"'))

#### Bootstrapping [20/40 minutes]: pretending a sample is the population

1. Why `replace=False`?
2. Why is `n` the same as the original sample size?

In [ ]:
keep = (df.origin=='Guatemala') | (df.origin=='Mexico')
px.histogram(df[keep], x='points', facet_col='origin', facet_col_wrap=2, height=300)

In [ ]:
contry = 'Mexico' 

# bootstrapping is when `replace=True` and `n` is the original sample size
# and we do this over and over to see the behavior of sample statistics
n_ = (df.origin==contry).sum() # ?
replace_ = True # ?

x = df[df.origin==contry].sample(n=n_, replace=replace_).points
print("The sample mean for the current sample is", x.mean()) 
# sample average "x-bar" a (sample) "statistic" (not a parameter)

dat = pd.DataFrame({'values': np.r_[df[df.origin==contry].points.values,x],
                    'sample': np.r_[['Orginal Sample']*(df.origin==contry).sum(),
                                    ['Bootstrap Sample']*n_]})             

fig = px.histogram(dat, x="values", color="sample", barmode="overlay")
fig.add_vline(x=x.mean(), line_dash="dot", annotation_text='Sample mean '+str(x.mean()))
fig.update_layout(yaxis_range=[0,30])
# Notice that we don't have a "Poulation Model"... only the "Original Sample"

In [ ]:
number_of_simulations = 1000 # adjust and experiment with this
simulated_means = np.zeros(number_of_simulations)

# np.random.seed(130) # ?
n = 100 # adjust and experiment with this

for i in range(number_of_simulations):
    # np.random.seed(130) # ?
    simulated_means[i] = df[df.origin==contry].sample(n=n_, replace=replace_).points.mean()

title = str(number_of_simulations)+' simulated means for sample of size n = '+str(n)
fig = px.histogram(pd.DataFrame({title: simulated_means}), x=title,
                   histnorm='probability density')    

support = np.linspace(simulated_means.min(),simulated_means.max(),100)
fig.add_trace(go.Scatter(x=support, y=stats.norm(0,scale=1/np.sqrt(n)).pdf(support), 
                         mode='lines', name='A theoretical<br>distribution of<br>"averages"'))

In [ ]:
number_of_simulations = 1000 # adjust and experiment with this
simulated_means = np.zeros(number_of_simulations)

# np.random.seed(130) # ?
n = (df.origin==contry).sum()

for i in range(number_of_simulations):
    simulated_means[i] = stats.norm(loc=0, scale=1).rvs(size=n).mean()

title = str(number_of_simulations)+' simulated means for sample of size n = '+str(n)
fig = px.histogram(pd.DataFrame({title: simulated_means}), x=title,
                   histnorm='probability density')    

support = np.linspace(simulated_means.min(),simulated_means.max(),100)
fig.add_trace(go.Scatter(x=support, y=stats.norm(0,scale=1/np.sqrt(n)).pdf(support), 
                         mode='lines', name='A theoretical<br>distribution of<br>"averages"'))

### Communication [40 minutes]

#### Activity 1 [20/40 minutes]

Break into 5 groups of students, assigning each group to one of the questions. Groups discuss questions for 5 minutes, and then each group (in order) provides their answer to the class for 3 minutes.

1. What are the differences between sampling from a "population model" (such as a normal or gamma distribution) compared to bootstrap sampling from an original sample? 

2. What happens to the variability of sample mean statistics when sampling from a "population model" (such as a normal or gamma distribution) as the sample size (n) increases?

3. Why does it make sense to consider changing the sample size when sampling from a "population model" (such as a normal or gamma distribution) but it does not make sense to change the sample size when considering bootstrapped samples from an original sample? 

4. If you had a histogram of bootstrapped sample means representing the variability of means that an observed sample of size n produces, how would you give a range estimating what the sample mean of a future sample of size n might be? 

5. If you had a theoretical distribution of "averages" representing the variability of means that an observed sample of size n produces, how would you give a range estimating what the sample mean of a future sample of size n might be? 


#### Activity 2 [20/40 minutes]

Break into 3 groups of students, assigning each group to one of the questions. Groups discuss questions for 5 minutes, and then each group (in order) provides their answer to the class for 5 minutes.

1. What is the process of bootstrapping?
2. What is the main purpose of bootstrapping?
3. If you had a hypothesis about what the average of a sample of size n from a population was, and then you observed the sample, how could you use bootstrapping as evidence in favor or against your hypothesis? 

